<a href="https://www.kaggle.com/code/elcientfico/driving-a-taxi-using-tree-methods-and-convolutions?scriptVersionId=148979515" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.layers import (Input,Conv2D,GlobalMaxPooling2D,Conv3D,
                                    GlobalMaxPooling3D)# Input data files are available in the readGlobalMaxPooling3D-only "../input/" directory
from tensorflow.keras.models import Model# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from PIL import Image
import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [18]:
im_size = Image.open("/kaggle/input/gta-san-andreas-driving-dataset-with-label/GTA_SanAndreasDriving/screenshots/1000_1010.png").size
im_size = (im_size[1],im_size[0],3)

In [19]:
class conv_model:
    def __init__(self):
        pass
def conv_model(ishape = (3,512,512,3), filters = 256, kernel_s = 3, strides = 3):
    x1  = Input(shape = ishape)
    x2 = Conv3D(filters,kernel_s,strides = 1)(x1)
#     x2 = Conv3D(filters,kernel_s,strides = 1)(x2)
#     if len(ishape)<3:
#         x2 = tf.expand_dims(x1,-1)
#         x2 = Conv2D(filters,kernel_s,strides = 1)(x2)
#     else:
#         x2 = Conv2D(filters,kernel_s,strides = 1)(x1)
#     x3 = Conv2D(filters,kernel_s,strides = 1)(x2)
    x4 = GlobalMaxPooling3D()(x2)
    model = Model(x1,x4)
    return model


In [20]:
n_steps = 6
input_shape = (n_steps,*im_size)
model = conv_model(input_shape,256,3,2)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 6, 600, 795, 3)]  0         
                                                                 
 conv3d_2 (Conv3D)           (None, 4, 598, 793, 256)  20992     
                                                                 
 global_max_pooling3d_2 (Glo  (None, 256)              0         
 balMaxPooling3D)                                                
                                                                 
Total params: 20,992
Trainable params: 20,992
Non-trainable params: 0
_________________________________________________________________


In [21]:
ss_path = "/kaggle/input/gta-san-andreas-driving-dataset-with-label/GTA_SanAndreasDriving/screenshots"
file_list = os.listdir(ss_path)
keys = ["W","S","A","D"]
keys_list = np.array([list(file_name.split("_")[1].split(".")[0]) for file_name in file_list]).astype(int)

LAod images

In [22]:
def get_batch(sub_file_list,ss_path,steps_n):
    steps_n = steps_n-1
    image_batch = []
    for idx,file_name in enumerate(sub_file_list):
        img = np.array(Image.open(os.path.join(ss_path,file_name)))
        image_batch.append(img)
    image_batch = np.stack(image_batch)
    image_batch = np.expand_dims(image_batch,1)
    aditional_arrays = None
    for step in range(steps_n):
        shifted_array = image_batch[:-step]
        zeros_array = np.zeros_like(image_batch[-step:])
        shifted_array = np.concatenate((zeros_array,shifted_array),0)
        assert(shifted_array.shape == image_batch.shape)
        if step == 0:
            aditional_arrays = shifted_array
        else:
            aditional_arrays = np.concatenate((aditional_arrays,shifted_array),1)
    image_batch = np.concatenate((image_batch,aditional_arrays),1)
    return image_batch

In [23]:
from tqdm import tqdm
enc_img_list = []
batch_size = 4
for batch_n in range(np.ceil(len(file_list)/batch_size).astype(int)):
    curr_range = slice((batch_n*batch_size),((batch_n+1)*batch_size))
    sub_file_list = file_list[curr_range]
    curr_batch = get_batch(sub_file_list,ss_path,n_steps)
    curr_pred = model.predict_on_batch(curr_batch)
    enc_img_list.append(curr_pred)
enc_img_list = np.concatenate(enc_img_list,0)
# for image_path in tqdm(file_list):
#     im = np.array(Image.open(os.path.join(ss_path,image_path)))
#     im = np.expand_dims(im/255,0)
#     conv_im = model.predict(im,verbose = False)
#     enc_img_list.append(conv_im)


In [24]:
enc_img_list = np.array(enc_img_list).squeeze()
keys_list = np.array(keys_list)

In [25]:
import xgboost as xgb

In [26]:
#["W","S","A","D"]

def mullab_to_mulclass(y):
    if all(y == [1,0,0,0]):
        return [1,0,0,0,0,0,0,0,0]
    elif all(y == [0,1,0,0]):
        return [0,1,0,0,0,0,0,0,0]
    elif all(y == [0,0,1,0]):
        return [0,0,1,0,0,0,0,0,0]
    elif all(y == [0,0,0,1]):
        return [0,0,0,1,0,0,0,0,0]
    elif all(y == [1,1,0,0]):
        return [0,0,0,0,1,0,0,0,0]
    elif all(y == [1,0,1,0]):
        return [0,0,0,0,0,1,0,0,0]
    elif all(y == [0,1,1,0]):
        return [0,0,0,0,0,0,1,0,0]
    elif all(y == [0,0,1,1]):
        return [0,0,0,0,0,0,0,1,0]
    else:
        return [0,0,0,0,0,0,0,0,1]
ylabels = []
for label in keys_list.astype(int):
    ylabels.append(mullab_to_mulclass(label))

In [27]:
prev_steps = 20
enc_img_list_plus_prev = np.array(enc_img_list)
for step in range(1,prev_steps+1):
    enc_img_list_0 = np.concatenate(
        (enc_img_list[:-step],np.zeros(shape = enc_img_list[:step,:].shape))
        ,0
    )
    enc_img_list_plus_prev = np.concatenate((enc_img_list_plus_prev,enc_img_list_0),axis = -1)

In [41]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(enc_img_list,keys_list,test_size=.2,stratify = keys_list) 

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
# rf = MultiOutputClassifier(HistGradientBoostingClassifier(early_stopping = True, verbose  = True,max_iter = 10000, learning_rate = 0.01,
#                                                          n_iter_no_change = 100, scoring = "f1",min_samples_leaf = 1, 
#                                                           max_leaf_nodes = 14))

    
early_stop = xgb.callback.EarlyStopping(
rounds=40, metric_name='logloss', save_best=True,
)

print("traing...")

rf = xgb.XGBClassifier(n_estimators = 10000,callbacks=[early_stop], learning_rate = .1, eval_metric = "logloss")

rf.fit(x_train,y_train,eval_set=[(x_test, y_test)], verbose = True)

print("Test Acc",rf.score(x_test,y_test),
"Train Acc",rf.score(x_train,y_train))

print("Test F1",f1_score(y_test,rf.predict(x_test),average = "micro"),
"Train F1",f1_score(y_train,rf.predict(x_train),average = "micro"))


traing...
[0]	validation_0-logloss:0.64635
[1]	validation_0-logloss:0.60870
[2]	validation_0-logloss:0.57715
[3]	validation_0-logloss:0.55143
[4]	validation_0-logloss:0.52988
[5]	validation_0-logloss:0.51129
[6]	validation_0-logloss:0.49559
[7]	validation_0-logloss:0.48188
[8]	validation_0-logloss:0.47077
[9]	validation_0-logloss:0.46119
[10]	validation_0-logloss:0.45333
[11]	validation_0-logloss:0.44604
[12]	validation_0-logloss:0.43980
[13]	validation_0-logloss:0.43460
[14]	validation_0-logloss:0.43015
[15]	validation_0-logloss:0.42633
[16]	validation_0-logloss:0.42319
[17]	validation_0-logloss:0.42055
[18]	validation_0-logloss:0.41802
[19]	validation_0-logloss:0.41602
[20]	validation_0-logloss:0.41441
[21]	validation_0-logloss:0.41256
[22]	validation_0-logloss:0.41131
[23]	validation_0-logloss:0.41013
[24]	validation_0-logloss:0.40957
[25]	validation_0-logloss:0.40836
[26]	validation_0-logloss:0.40779
[27]	validation_0-logloss:0.40724
[28]	validation_0-logloss:0.40723
[29]	validatio

In [30]:
from PIL import Image, ImageDraw, ImageFont
import cv2
import os

# Directory containing the input images
image_dir = "input_images/"

# List of image file names in the directory
image_files = sorted(os.listdir(image_dir))

# Output video file name
output_video = "output_video.mp4"

# Text to overlay and its color
text = "Hello, World!"
text_color = (255, 0, 0)  # Red color (R, G, B)

# Create a drawing context
font = ImageFont.load_default()

# Video settings
frame_rate = 30
frame_size = im_size # Adjust the size as needed
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Video codec

# Initialize the video writer
out = cv2.VideoWriter(output_video, fourcc, frame_rate, frame_size)

# Iterate through the images and add text, then write to the video
for image_file in image_files:
    # Load the image
    image = Image.open(os.path.join(image_dir, image_file))

    # Create a drawing context
    draw = ImageDraw.Draw(image)

    # Specify the position for the text (adjust as needed)
    position = (im_size[0]/2, 8*im_size[1]/9)

    # Draw the text on the image
    draw.text(position, text, fill=text_color, font=font,align = "center")

    # Convert the Pillow image to a NumPy array (OpenCV format)
    image_np = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Write the frame to the video
    out.write(image_np)

# Release the video writer
out.release()

print(f"Video saved as {output_video}")


FileNotFoundError: [Errno 2] No such file or directory: 'input_images/'